# PROJECT: RETAILS TRANSACTIONS: FIND THE BEST CHURNED CUSTOMER TO SEND COUPON

# INTRODUCTION

The dataset contains information of customers in Floormart, a large retails company.
We are data analyst in the marking department. The manager wants to  use the remaining budget 1,000USD to convert some physical store customers to the online store by sending them coupons to use online. This customer must be the best customer that purchased the most but no transaction the recent 3 months.

The task: Find the best customers (name and address) to send coupon to. How many people and what amount of coupon to maximize the benefit?

In [86]:
#Read and overview the dataset
import os

os.getcwd()
os.chdir("D:\PERSONAL\CODING\PROJECT PYTHON\RETAILS TRANSACTION")
import numpy as np
import pandas as pd
import datetime as dt
data=pd.read_csv("Retail_Data_Transactions.csv")
print(data.head(20))

   customer_id trans_date  tran_amount
0       CS5295  11-Feb-13           35
1       CS4768  15-Mar-15           39
2       CS2122  26-Feb-13           52
3       CS1217  16-Nov-11           99
4       CS1850  20-Nov-13           78
5       CS5539  26-Mar-14           81
6       CS2724  06-Feb-12           93
7       CS5902  30-Jan-15           89
8       CS6040  08-Jan-13           76
9       CS3802  20-Aug-13           75
10      CS3494  02-Jul-13           94
11      CS3780  25-Mar-13           80
12      CS1171  03-Nov-12           59
13      CS2892  12-May-13           43
14      CS5552  29-Dec-14           78
15      CS6043  15-Jan-14           98
16      CS4147  08-Jul-13           81
17      CS4655  30-Dec-13           93
18      CS3904  20-Jul-14          103
19      CS4102  09-Jul-11           96


In [87]:
print(data.describe())

         tran_amount
count  125000.000000
mean       64.991912
std        22.860006
min        10.000000
25%        47.000000
50%        65.000000
75%        83.000000
max       105.000000


In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


# Data cleaning and exploration

In [89]:
#The column trans_date should be datetime type
data["trans_date"]=pd.to_datetime(data["trans_date"])
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB
None


In [90]:
print(data["trans_date"].min())
print(data["trans_date"].max())

2011-05-16 00:00:00
2015-03-16 00:00:00


It seems the data is cleaned and not missing but the data contains the transaction since 2012. In order to answer the question, we should only chose customer that made purchased within 2014-2015

In [91]:
min_date=dt.datetime(2014,1,1)
data=data[data["trans_date"]>= min_date]
data

,customer_id,trans_date,tran_amount
1,CS4768,2015-03-15,39
5,CS5539,2014-03-26,81
7,CS5902,2015-01-30,89
14,CS5552,2014-12-29,78
15,CS6043,2014-01-15,98
...,...,...,...
124990,CS8960,2014-07-04,39
124991,CS8076,2014-05-16,68
124996,CS7232,2014-08-19,38
124997,CS8731,2014-11-28,42


# Data analyzing

In [92]:
#Groupby customer by customer_id
group_by_customer=data.groupby("customer_id")

#Find the last time customer purchased at the Floormart store:
last_transaction=group_by_customer["trans_date"].max()
best_churned=pd.DataFrame(last_transaction)

#Create the column to find the amount that each customer spent at the Floormart store:
best_churned["amount_spent"]=group_by_customer.sum()
best_churned

#Create the column to define whether customer is churned or not.
#Churned customer is defined by the fact that they haven't made any purchases in recent 3 months
cutoff_day=dt.datetime(year=2014, month = 12, day =16)
def check_date(date):
    if date < cutoff_day:
        return 1
    else:
        return 0
best_churned["churned"]=best_churned["trans_date"].apply(check_date)

#create the column of number of transactions:

best_churned["no_trans"]=data["customer_id"].value_counts(sort=True)

best_churned

,trans_date,amount_spent,churned,no_trans
customer_id,,,,
CS1112,2015-01-14,251,0,4
CS1113,2015-02-09,446,0,7
CS1114,2015-02-12,504,0,7
CS1115,2015-03-05,368,0,5
CS1116,2014-08-25,333,1,5
...,...,...,...,...
CS8996,2014-12-09,336,1,6
CS8997,2014-06-28,84,1,2
CS8998,2014-12-22,130,0,4


The best_churned is the final dataset that we can analyze to find the best customer. Due to time constrainted, we will use the simple method "weighted sum model" to find the answer

In [93]:
#create a new column name scaled_tran
best_churned["scaled_tran"]=best_churned["no_trans"].min()
min_tran=best_churned["no_trans"].min()
max_tran=best_churned["no_trans"].max()
best_churned["scaled_tran"]=(best_churned["no_trans"]-min_tran)/(max_tran-min_tran)
min_amount=best_churned["amount_spent"].min()
max_amount=best_churned["amount_spent"].max()
best_churned["scaled_amount"]=(best_churned["amount_spent"]-min_amount)/(max_amount-min_amount)
#Create a new colum score to find the best customer
best_churned["score"]=(best_churned["scaled_amount"]+best_churned["scaled_tran"])/2*100
best_churned.sort_values("score", ascending=False, inplace=True)
best_churned

,trans_date,amount_spent,churned,no_trans,scaled_tran,scaled_amount,score
customer_id,,,,,,,
CS5244,2015-02-28,1162,0,17,1.0000,1.000000,100.000000
CS3270,2015-03-13,1145,0,16,0.9375,0.985243,96.137153
CS2647,2015-03-13,1158,0,14,0.8125,0.996528,90.451389
CS4074,2014-12-05,1027,1,15,0.8750,0.882812,87.890625
CS3426,2015-01-29,1022,0,15,0.8750,0.878472,87.673611
...,...,...,...,...,...,...,...
CS7956,2014-06-18,12,1,1,0.0000,0.001736,0.086806
CS7288,2014-07-08,12,1,1,0.0000,0.001736,0.086806
CS7594,2014-12-25,11,0,1,0.0000,0.000868,0.043403


In [94]:
#Choose the churned customers only:
best_churned=best_churned[best_churned["churned"]==1]
best_churned

,trans_date,amount_spent,churned,no_trans,scaled_tran,scaled_amount,score
customer_id,,,,,,,
CS4074,2014-12-05,1027,1,15,0.8750,0.882812,87.890625
CS2459,2014-12-03,945,1,13,0.7500,0.811632,78.081597
CS5057,2014-12-02,974,1,12,0.6875,0.836806,76.215278
CS3799,2014-10-16,900,1,13,0.7500,0.772569,76.128472
CS5390,2014-12-14,895,1,13,0.7500,0.768229,75.911458
...,...,...,...,...,...,...,...
CS7129,2014-02-27,14,1,1,0.0000,0.003472,0.173611
CS7956,2014-06-18,12,1,1,0.0000,0.001736,0.086806
CS7288,2014-07-08,12,1,1,0.0000,0.001736,0.086806


We have the budget 1,000USD so how many churned customer we should choose and how much coupon should be?
Based on the market, the coupon 30% is the good enough to prompt people to actually use them

In [95]:
coupon=data["tran_amount"].mean() * 0.3
coupon

19.51062806900275

In [96]:
number_of_coupons=1000/coupon
number_of_coupons

51.25411629309549

Rather than choose coupon 19USD for 51 customers we should choose 20USD coupon for 50 people instead

In [97]:
top_50_churned=best_churned.head(50)
top_50_churned

,trans_date,amount_spent,churned,no_trans,scaled_tran,scaled_amount,score
customer_id,,,,,,,
CS4074,2014-12-05,1027,1,15,0.8750,0.882812,87.890625
CS2459,2014-12-03,945,1,13,0.7500,0.811632,78.081597
CS5057,2014-12-02,974,1,12,0.6875,0.836806,76.215278
CS3799,2014-10-16,900,1,13,0.7500,0.772569,76.128472
CS5390,2014-12-14,895,1,13,0.7500,0.768229,75.911458
CS4163,2014-11-09,949,1,12,0.6875,0.815104,75.130208
CS3011,2014-12-14,858,1,13,0.7500,0.736111,74.305556
CS2996,2014-12-09,848,1,13,0.7500,0.727431,73.871528
CS3227,2014-11-30,919,1,12,0.6875,0.789062,73.828125


In [98]:
top_50_churned.to_csv("best_customers.txn")